In [78]:
import pandas as pd
import xgboost as xgb

In [79]:
# 1. Charger les données nécessaires
# Charger les équipes (nécessaire pour s'assurer que l'encodage des équipes de l'entrainement est le même)
team_categories = pd.read_csv('data/teams.csv')
# Convertir les équipes en catégories
team_categories = team_categories['team'].astype('category')

# Charger les joueurs (nécessaire car les scores des joueurs sont calculés dans le préprocessing)
try:
    players_scores = pd.read_csv('data/players_scores.csv', delimiter=';', encoding='utf-8')  # Délimiteur ; pour players_scores
    print("Colonnes de players_scores :", players_scores.columns.tolist())  # Débogage
except Exception as e:
    print(f"Erreur lors du chargement de players_scores.csv : {e}")
    raise

# Charger les matchs (nécessaire car les valeurs heads-to-heads sont calculées dans la prédiction)
try:
    schedule = pd.read_csv('data/schedule.csv', encoding='utf-8')  # Pas de délimiteur forcé, utiliser par défaut (virgule)
    print("Colonnes de schedule :", schedule.columns.tolist())  # Débogage
    if 'home_team' not in schedule.columns or 'away_team' not in schedule.columns:
        raise ValueError("Colonnes 'home_team' ou 'away_team' absentes dans schedule.csv")
except Exception as e:
    print(f"Erreur lors du chargement de schedule.csv : {e}")
    raise

# Charger les modèles (provenant du script d'entraînement training.ipynb)
home_score_model = xgb.XGBRegressor()
home_score_model.load_model('data/home_score_model.json')
away_score_model = xgb.XGBRegressor()
away_score_model.load_model('data/away_score_model.json')

Colonnes de players_scores : ['Player', 'Team', 'Pos', 'player_score', 'Player_URL']
Colonnes de schedule : ['game_id', 'home_team', 'away_team', 'home_score', 'away_score', 'date', 'away_xg', 'home_xg']


In [80]:
# 2. Définir les fonctions nécessaires pour prédire les scores

# 2.1 Fonction pour calculer la différence de buts et de xG sur les dernières confrontations directes, par rapport à l'équipe à domicile
def calculate_last_h2h_stats(home_team, away_team, schedule, nb_matches=3):
    # Filtrer les matchs entre les deux équipes
    h2h_matches = schedule[((schedule['home_team'] == home_team) & (schedule['away_team'] == away_team)) |
                           ((schedule['home_team'] == away_team) & (schedule['away_team'] == home_team))]

    # Trier par date pour obtenir les derniers matchs
    h2h_matches = h2h_matches.sort_index(ascending=False).head(nb_matches)

    # Calculer la différence de buts et de xG par rapport à l'équipe à domicile
    # Inverser les colonnes home_score et away_score si l'équipe à domicile est l'équipe visiteuse
    for index, row in h2h_matches.iterrows():
        if row['home_team'] == away_team:
            h2h_matches.at[index, 'home_score'], h2h_matches.at[index, 'away_score'] = row['away_score'], row['home_score']
            h2h_matches.at[index, 'home_xg'], h2h_matches.at[index, 'away_xg'] = row['away_xg'], row['home_xg']

    ## Calculer la différence de buts et de xG
    goal_diff = h2h_matches['home_score'] - h2h_matches['away_score']
    xg_diff = h2h_matches['home_xg'] - h2h_matches['away_xg']

    # Garder la somme des différences de buts et de xG
    if len(goal_diff) > 0 and len(xg_diff) > 0:
        goal_diff_sum = goal_diff.sum()
        xg_diff_sum = xg_diff.sum()
    else:
        goal_diff_sum = pd.NA
        xg_diff_sum = pd.NA

    return goal_diff, xg_diff, goal_diff_sum, xg_diff_sum

# 2.2 Fonction pour calculer le scores des joueurs pour chaque match
# Fonction pour calculer le score total de l'équipe selon les joueurs choisis
def calculate_team_score(team, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11):
    missing_players = []
    # Filtrer les joueurs de l'équipe
    print(f"Débogage : Vérification de l'équipe {team} dans players_scores")
    if 'Team' not in players_scores.columns:
        print("Erreur : Colonne 'Team' absente dans players_scores")
        return 0, missing_players
    team_df = players_scores[players_scores['Team'] == team]
    # Sélectionner les joueurs par leur nom
    selected_players = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11]
    # Filtrer les joueurs sélectionnés
    team_df = team_df[team_df['Player'].isin(selected_players)]
    # Vérifier si un joueur n'est pas trouvé
    if team_df.shape[0] < 11:
        missing_players = set(selected_players) - set(team_df['Player'])
        print(f"Attention : Joueurs manquants pour {team} : {missing_players}")
        for player in missing_players:
            found = players_scores[players_scores['Player'] == player]
            if not found.empty:
                team_df = pd.concat([team_df, found], ignore_index=True)
        missing_players = set(selected_players) - set(team_df['Player'])
    if team_df.shape[0] < 11:
        average_score = players_scores['player_score'].mean()
        for player in missing_players:
            team_df = pd.concat([team_df, pd.DataFrame({
                'Player': [player],
                'Team': [team],
                'player_score': [average_score]
            })], ignore_index=True)
    team_score = team_df['player_score'].sum()
    return team_score, missing_players

In [81]:
## 3. Fonction pour faire une prédiction pour un match specifique
def get_match_result(home_team, home_p1, home_p2, home_p3, home_p4, home_p5, home_p6, home_p7, home_p8, home_p9, home_p10, home_p11,
                   away_team, away_p1, away_p2, away_p3, away_p4, away_p5, away_p6, away_p7, away_p8, away_p9, away_p10, away_p11):
    # Préparer les données pour la prédiction
    X_pred = pd.DataFrame({
        'home_team': [home_team],
        'away_team': [away_team]})
    
    # Appliquer les niveaux des catégories de l’entraînement, si l’équipe n’est pas dans le DataFrame d’entraînement, retourner une erreur
    if home_team not in list(team_categories) or away_team not in list(team_categories):
        print(f"Erreur : L'équipe {home_team} ou {away_team} n'est pas dans les équipes connues.")
        print('home', home_team, 'away:', away_team)
        return pd.NA, pd.NA
    X_pred['home_team'] = pd.Categorical(X_pred['home_team'], categories=team_categories)
    X_pred['away_team'] = pd.Categorical(X_pred['away_team'], categories=team_categories)
    
    # 1ère étape : Extraire les features, ici historiques de confrontations directes des équipes
    # Calculer les différences de buts et de xG
    goal_diff, xg_diff, goal_diff_sum, xg_diff_sum = calculate_last_h2h_stats(home_team, away_team, schedule, nb_matches=3)
    # Stocker les résultats dans le DataFrame final
    X_pred['h2h_goal_diff_sum'] = goal_diff_sum
    X_pred['h2h_xg_diff_sum'] = xg_diff_sum

    # Calculer les scores selon les joueurs de chaque équipe
    home_players = [home_p1, home_p2, home_p3, home_p4, home_p5, home_p6, home_p7, home_p8, home_p9, home_p10, home_p11]
    away_players = [away_p1, away_p2, away_p3, away_p4, away_p5, away_p6, away_p7, away_p8, away_p9, away_p10, away_p11]
    # Calculer les scores des joueurs pour chaque équipe
    home_team_players_score, home_missing_players = calculate_team_score(home_team, *home_players)
    away_team_players_score, away_missing_players = calculate_team_score(away_team, *away_players)
    # S'il manque au moins 1 joueur dans une équipe, informer l'utilisateur
    if len(home_missing_players) > 0 or len(away_missing_players) > 0:
        print(f"Avertissement : Les joueurs suivants sont manquants pour le match {home_team} vs {away_team}, un score moyen leur est attribué :")
        print(home_missing_players, away_missing_players)
    # Stocker les scores dans le DataFrame final
    X_pred['home_team_players_score'] = home_team_players_score
    X_pred['away_team_players_score'] = away_team_players_score
    
    # Convertir les colonnes numériques de type 'object' en float
    X_pred['h2h_goal_diff_sum'] = pd.to_numeric(X_pred['h2h_goal_diff_sum'], errors='coerce')
    X_pred['h2h_xg_diff_sum'] = pd.to_numeric(X_pred['h2h_xg_diff_sum'], errors='coerce')
    X_pred['home_team_players_score'] = pd.to_numeric(X_pred['home_team_players_score'], errors='coerce')
    X_pred['away_team_players_score'] = pd.to_numeric(X_pred['away_team_players_score'], errors='coerce')

    # 2ème étape : Prédire les scores
    home_goals = home_score_model.predict(X_pred)
    away_goals = away_score_model.predict(X_pred)
    # récupérer le résultat
    return (round(float(home_goals.item()),1), round(float(away_goals.item())))

In [82]:
# Exemple d'utilisation de la fonction pour prédire le score d'un match
home_team = 'Manchester City'
home_players = [
    'Ederson', 'Rúben Dias', 'Joško Gvardiol', 'Rico Lewis', 'Bernardo Silva',
    'İlkay Gündoğan', 'Mateo Kovačić', 'Phil Foden', 'Erling Haaland', 'Vitor Reis', 'John Stones'  # Remplacement de None par Nathan Aké (exemple)
]
away_team = 'Liverpool'
away_players = [
    'Vitezslav Jaros', 'Federico Chiesa', 'Virgil van Dijk', 'Andrew Robertson', 'Ibrahima Konaté',
    'Ryan Gravenberch', 'Alexis Mac Allister', 'Dominik Szoboszlai', 'Jayden Danns', 'Cody Gakpo', 'Luis Díaz'
]
home_pred, away_pred = get_match_result(home_team, *home_players, away_team, *away_players)
print(f"{home_team} {home_pred} - {away_pred} {away_team}")

Débogage : Vérification de l'équipe Manchester City dans players_scores
Débogage : Vérification de l'équipe Liverpool dans players_scores
Manchester City 0.4 - 1 Liverpool
